Flight Difficulty Score Development

In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [174]:
# Set seaborn style
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.1)
plt.rcParams['figure.figsize'] = (14, 6)

In [175]:
airports = pd.read_csv("data/Airports Data.csv")
bag_level = pd.read_csv("data/Bag+Level+Data.csv")
flight_level = pd.read_csv("data/Flight Level Data.csv")
pnr_remark_level = pd.read_csv("data/PNR Remark Level Data.csv")
pnr_flight_level = pd.read_csv("data/PNR+Flight+Level+Data.csv")

In [176]:
# Convert datetime columns
flight_level['scheduled_departure_datetime_local'] = pd.to_datetime(flight_level['scheduled_departure_datetime_local'])
flight_level['actual_departure_datetime_local'] = pd.to_datetime(flight_level['actual_departure_datetime_local'])
flight_level['scheduled_departure_date_local'] = pd.to_datetime(flight_level['scheduled_departure_date_local'])
bag_level['scheduled_departure_date_local'] = pd.to_datetime(bag_level['scheduled_departure_date_local'])
pnr_flight_level['scheduled_departure_date_local'] = pd.to_datetime(pnr_flight_level['scheduled_departure_date_local'])

In [177]:
# Remove negative or zero ground times
invalid_ground_time = (flight_level['scheduled_ground_time_minutes'] <= 0) | \
                      (flight_level['minimum_turn_minutes'] <= 0)
print(f"  Rows with invalid ground time: {invalid_ground_time.sum():,}")
flight_level = flight_level[~invalid_ground_time].copy()

  Rows with invalid ground time: 312


In [178]:
# Calculate delay
flight_level['delay_minutes'] = (
    flight_level['actual_departure_datetime_local'] - 
    flight_level['scheduled_departure_datetime_local']
).dt.total_seconds() / 60

In [179]:
# Create flight key
flight_key = ['company_id', 'flight_number', 'scheduled_departure_date_local',
              'scheduled_departure_station_code', 'scheduled_arrival_station_code']

# Start with flight level data
df_master = flight_level.copy()


In [180]:
df_master['ground_time_ratio'] = (
    df_master['scheduled_ground_time_minutes'] / df_master['minimum_turn_minutes']
)

df_master['ground_time_pressure_score'] = np.where(
    df_master['ground_time_ratio'] < 1.0, 10,      # Critical - below minimum
    np.where(df_master['ground_time_ratio'] < 1.1, 7,  # Very tight
             np.where(df_master['ground_time_ratio'] < 1.3, 4, 1))  # Tight / Adequate
)

In [181]:
bag_summary = bag_level.groupby(flight_key).agg({
    'bag_tag_unique_number': 'count',
    'bag_type': lambda x: (x == 'Transfer').sum()
}).reset_index()
bag_summary.columns = list(bag_summary.columns[:-2]) + ['total_bags', 'transfer_bags']
bag_summary['checked_bags'] = bag_summary['total_bags'] - bag_summary['transfer_bags']
bag_summary['transfer_ratio'] = bag_summary['transfer_bags'] / bag_summary['total_bags']

df_master = df_master.merge(bag_summary, on=flight_key, how='left')
df_master['total_bags'] = df_master['total_bags'].fillna(0)
df_master['transfer_ratio'] = df_master['transfer_ratio'].fillna(0)


In [182]:
pnr_pax = pnr_flight_level.groupby(flight_key).agg({
    'total_pax': 'sum',
    'lap_child_count': 'sum',
    'is_child': 'sum',
    'basic_economy_ind': 'sum',
    'is_stroller_user': 'sum'
}).reset_index()

df_master = df_master.merge(pnr_pax, on=flight_key, how='left')
df_master['total_pax'].fillna(0, inplace=True)
df_master['load_factor'] = df_master['total_pax'] / df_master['total_seats']

# Load factor score - REDUCED WEIGHT based on negative correlation with delay
# Analysis shows load factor has -0.25 correlation with delay
# Only very high loads (>95%) might cause minor issues with boarding
df_master['load_factor_score'] = np.where(
    df_master['load_factor'] > 0.95, 3,  # Reduced from 10
    np.where(df_master['load_factor'] > 0.90, 2,  # Reduced from 7
             np.where(df_master['load_factor'] > 0.85, 1, 0))  # Reduced from 4
)

# Bags per passenger
df_master['bags_per_pax'] = df_master['total_bags'] / df_master['total_pax'].replace(0, 1)
df_master['baggage_volume_score'] = np.where(
    df_master['bags_per_pax'] > 1.5, 8,
    np.where(df_master['bags_per_pax'] > 1.2, 5,
             np.where(df_master['bags_per_pax'] > 0.8, 2, 0))
)

# Transfer bag complexity
df_master['transfer_complexity_score'] = np.where(
    df_master['transfer_ratio'] > 0.6, 8,
    np.where(df_master['transfer_ratio'] > 0.4, 5,
             np.where(df_master['transfer_ratio'] > 0.2, 2, 0))
)

/tmp/ipykernel_86676/1488398904.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_master['total_pax'].fillna(0, inplace=True)


In [183]:
# First, convert Y/N to 1/0, then aggregate
flight_key = ['company_id', 'flight_number', 'scheduled_departure_date_local',
              'scheduled_departure_station_code', 'scheduled_arrival_station_code']

# Create numeric versions in pnr_flight_level
pnr_flight_level['is_child_numeric'] = (pnr_flight_level['is_child'] == 'Y').astype(int)
pnr_flight_level['is_stroller_numeric'] = (pnr_flight_level['is_stroller_user'] == 'Y').astype(int)
pnr_flight_level['lap_child_count'] = pd.to_numeric(pnr_flight_level['lap_child_count'], errors='coerce').fillna(0)
pnr_flight_level['total_pax'] = pd.to_numeric(pnr_flight_level['total_pax'], errors='coerce').fillna(0)

# Aggregate to flight level
pnr_aggregated = pnr_flight_level.groupby(flight_key).agg({
    'is_child_numeric': 'sum',      # Total children on flight
    'lap_child_count': 'sum',       # Total lap children
    'is_stroller_numeric': 'sum',   # Total stroller users
    'total_pax': 'sum',             # Total passengers from all PNRs
    'record_locator': 'nunique'     # Number of unique PNRs (family groups)
}).reset_index()

# Rename for clarity
pnr_aggregated.rename(columns={
    'is_child_numeric': 'total_children',
    'lap_child_count': 'total_lap_children',
    'is_stroller_numeric': 'total_stroller_users',
    'total_pax': 'total_pax_from_pnr',
    'record_locator': 'num_pnr_groups'
}, inplace=True)

# Merge with flight_level data
# df_master = flight_level.merge(pnr_aggregated, on=flight_key, how='left')
df_master = df_master.merge(pnr_aggregated, on=flight_key, how='left')

# Fill NaN values
df_master['total_children'] = df_master['total_children'].fillna(0)
df_master['total_lap_children'] = df_master['total_lap_children'].fillna(0)
df_master['total_stroller_users'] = df_master['total_stroller_users'].fillna(0)
df_master['total_pax_from_pnr'] = df_master['total_pax_from_pnr'].fillna(0)
df_master['num_pnr_groups'] = df_master['num_pnr_groups'].fillna(0)

# Create enhanced features
df_master['child_complexity'] = (
    df_master['total_children'] + 
    df_master['total_lap_children'] * 1.5 +  # Lap children are more complex
    df_master['total_stroller_users'] * 1.2   # Strollers add complexity
)

# Average PNR size (larger groups = families)
df_master['avg_pnr_size'] = np.where(
    df_master['num_pnr_groups'] > 0,
    df_master['total_pax_from_pnr'] / df_master['num_pnr_groups'],
    0
)

df_master['family_complexity_score'] = np.where(
    df_master['child_complexity'] > 15, 6,
    np.where(df_master['child_complexity'] > 10, 4,
             np.where(df_master['child_complexity'] > 5, 2, 0))
)

In [184]:
ssr_counts = pnr_remark_level.groupby('flight_number').size().reset_index(name='ssr_count')
df_master = df_master.merge(ssr_counts, on='flight_number', how='left')
df_master['ssr_count'].fillna(0, inplace=True)

df_master['ssr_score'] = np.where(
    df_master['ssr_count'] > 15, 7,
    np.where(df_master['ssr_count'] > 10, 5,
             np.where(df_master['ssr_count'] > 5, 3, 0))
)

/tmp/ipykernel_86676/1882751893.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_master['ssr_count'].fillna(0, inplace=True)


In [185]:
# Wide-body aircraft
df_master['is_wide_body'] = df_master['fleet_type'].str.contains('787|777|747|767', na=False).astype(int)
df_master['aircraft_complexity_score'] = df_master['is_wide_body'] * 3

# Time of day
df_master['hour'] = df_master['scheduled_departure_datetime_local'].dt.hour
df_master['is_peak_time'] = (
    (df_master['hour'].between(6, 9)) | (df_master['hour'].between(16, 19))
).astype(int)
df_master['peak_time_score'] = df_master['is_peak_time'] * 2

# International flights
df_master = df_master.merge(
    airports.rename(columns={
        'airport_iata_code': 'scheduled_arrival_station_code',
        'iso_country_code': 'arrival_country'
    }),
    on='scheduled_arrival_station_code',
    how='left'
)
df_master['is_international'] = (df_master['arrival_country'] != 'US').astype(int)
df_master['international_score'] = df_master['is_international'] * 4

In [186]:
df_master['has_delay'] = (df_master['delay_minutes'] > 0).astype(int)
df_master['delay_severity_score'] = np.where(
    df_master['delay_minutes'] > 30, 5,
    np.where(df_master['delay_minutes'] > 15, 3,
             np.where(df_master['delay_minutes'] > 0, 1, 0))
)


In [187]:
df_master['difficulty_score'] = (
    df_master['ground_time_pressure_score'] * 1.0 +      # Critical operational constraint
    df_master['load_factor_score'] * 0.9 +               # High loads = boarding delays
    df_master['baggage_volume_score'] * 0.8 +            # Baggage handling time
    df_master['transfer_complexity_score'] * 0.8 +       # Transfer coordination
    df_master['ssr_score'] * 0.7 +                       # Special assistance
    df_master['family_complexity_score'] * 0.6 +         # Family boarding complexity
    df_master['international_score'] * 0.5 +             # Documentation/customs
    df_master['aircraft_complexity_score'] * 0.4 +       # Aircraft size
    df_master['peak_time_score'] * 0.3 +                 # Resource congestion
    df_master['delay_severity_score'] * 0.2              # Historical pattern (low weight)
)

In [188]:
df_master.sample(10)

,company_id,flight_number,scheduled_departure_date_local,scheduled_departure_station_code,scheduled_arrival_station_code,scheduled_departure_datetime_local,scheduled_arrival_datetime_local,actual_departure_datetime_local,actual_arrival_datetime_local,total_seats,...,aircraft_complexity_score,hour,is_peak_time,peak_time_score,arrival_country,is_international,international_score,has_delay,delay_severity_score,difficulty_score
5677,UA,422,2025-08-05,ORD,ATH,2025-08-05 16:15:00+00:00,2025-08-06T10:30:00Z,2025-08-05 16:21:00+00:00,2025-08-06T10:55:00Z,318,...,3,16,1,2,GR,1,4,1,1,20.2
4994,UA,2418,2025-08-04,ORD,PHL,2025-08-04 07:35:00+00:00,2025-08-04T10:47:00Z,2025-08-04 07:30:00+00:00,2025-08-04T10:26:00Z,150,...,0,7,1,2,US,0,0,0,0,8.3
2637,OO,5505,2025-08-13,ORD,SBN,2025-08-13 15:40:00+00:00,2025-08-13T17:39:00Z,2025-08-13 15:42:00+00:00,2025-08-13T17:43:00Z,50,...,0,15,0,0,US,0,0,1,1,9.3
280,YX,3614,2025-08-12,ORD,MCI,2025-08-12 21:25:00+00:00,2025-08-12T23:14:00Z,2025-08-12 23:12:00+00:00,2025-08-13T01:04:00Z,70,...,0,21,0,0,US,0,0,1,5,13.3
7639,UA,980,2025-08-05,ORD,DUB,2025-08-05 20:15:00+00:00,2025-08-06T09:45:00Z,2025-08-05 20:20:00+00:00,2025-08-06T09:34:00Z,243,...,3,20,0,0,IE,1,4,1,1,16.3
1652,UA,342,2025-08-02,ORD,GCM,2025-08-02 09:26:00+00:00,2025-08-02T13:30:00Z,2025-08-02 09:18:00+00:00,2025-08-02T14:23:00Z,179,...,0,9,1,2,KY,1,4,0,0,14.9
3711,UA,286,2025-08-07,ORD,CLE,2025-08-07 07:33:00+00:00,2025-08-07T09:59:00Z,2025-08-07 07:29:00+00:00,2025-08-07T09:41:00Z,179,...,0,7,1,2,US,0,0,0,0,16.8
2349,UA,2387,2025-08-08,ORD,MCO,2025-08-08 06:00:00+00:00,2025-08-08T09:47:00Z,2025-08-08 05:47:00+00:00,2025-08-08T09:35:00Z,179,...,0,6,1,2,US,0,0,0,0,16.8
6982,UA,1583,2025-08-07,ORD,BWI,2025-08-07 18:35:00+00:00,2025-08-07T21:42:00Z,2025-08-07 20:32:00+00:00,2025-08-07T23:29:00Z,179,...,0,18,1,2,US,0,0,1,5,20.8
4221,UA,980,2025-08-07,ORD,DUB,2025-08-07 20:15:00+00:00,2025-08-08T09:45:00Z,2025-08-07 20:36:00+00:00,2025-08-08T10:24:00Z,318,...,3,20,0,0,IE,1,4,1,3,20.0


In [109]:
flight_level.sample(10)

,company_id,flight_number,scheduled_departure_date_local,scheduled_departure_station_code,scheduled_arrival_station_code,scheduled_departure_datetime_local,scheduled_arrival_datetime_local,actual_departure_datetime_local,actual_arrival_datetime_local,total_seats,fleet_type,carrier,scheduled_ground_time_minutes,actual_ground_time_minutes,minimum_turn_minutes,delay_minutes
3572,YX,3598,2025-08-05,ORD,SDF,2025-08-05 17:55:00+00:00,2025-08-05T20:26:00Z,2025-08-05 17:52:00+00:00,2025-08-05T20:12:00Z,76,ERJ-175,Express,65,50,34,-3.0
5470,G7,4517,2025-08-05,ORD,SCE,2025-08-05 14:10:00+00:00,2025-08-05T17:01:00Z,2025-08-05 14:05:00+00:00,2025-08-05T17:09:00Z,50,CRJ-550,Express,55,51,29,-5.0
7129,UA,1316,2025-08-15,ORD,SRQ,2025-08-15 11:14:00+00:00,2025-08-15T15:10:00Z,2025-08-15 11:04:00+00:00,2025-08-15T14:57:00Z,126,B737-700,Mainline,105,112,43,-10.0
4937,OO,4645,2025-08-09,ORD,FWA,2025-08-09 11:05:00+00:00,2025-08-09T13:17:00Z,2025-08-09 11:19:00+00:00,2025-08-09T13:14:00Z,50,CRJ-200,Express,106,122,29,14.0
4758,UA,587,2025-08-14,ORD,YUL,2025-08-14 14:07:00+00:00,2025-08-14T17:26:00Z,2025-08-14 14:26:00+00:00,2025-08-14T17:34:00Z,166,B737-MAX8,Mainline,73,87,65,19.0
4070,OO,5624,2025-08-08,ORD,LEX,2025-08-08 17:50:00+00:00,2025-08-08T20:21:00Z,2025-08-08 17:48:00+00:00,2025-08-08T20:14:00Z,76,ERJ-175,Express,65,61,34,-2.0
3514,OO,5225,2025-08-01,ORD,MKE,2025-08-01 16:00:00+00:00,2025-08-01T17:00:00Z,2025-08-01 15:56:00+00:00,2025-08-01T17:04:00Z,50,CRJ-200,Express,40,31,29,-4.0
6020,UA,791,2025-08-02,ORD,DEN,2025-08-02 07:10:00+00:00,2025-08-02T08:58:00Z,2025-08-02 07:17:00+00:00,2025-08-02T08:48:00Z,200,A321-2NX,Mainline,55,65,68,7.0
4120,UA,600,2025-08-15,ORD,LAS,2025-08-15 14:40:00+00:00,2025-08-15T16:40:00Z,2025-08-15 14:35:00+00:00,2025-08-15T16:33:00Z,179,B737-900,Mainline,86,99,56,-5.0
4661,OO,5260,2025-08-04,ORD,FSD,2025-08-04 08:54:00+00:00,2025-08-04T10:50:00Z,2025-08-04 08:58:00+00:00,2025-08-04T11:14:00Z,50,CRJ-550,Express,68,88,29,4.0


In [96]:
pnr_flight_level.head()

,company_id,flight_number,scheduled_departure_date_local,scheduled_departure_station_code,scheduled_arrival_station_code,record_locator,pnr_creation_date,total_pax,is_child,basic_economy_ind,is_stroller_user,lap_child_count
0,UA,2494,2025-08-04,ORD,MCI,PNR_520583,2025-07-07,1,N,0,N,0
1,UA,2483,2025-08-06,ORD,IAH,PNR_296107,2025-03-28,1,N,0,N,0
2,UA,1620,2025-08-01,ORD,SEA,PNR_296108,2025-06-30,4,Y,0,N,0
3,UA,1620,2025-08-01,ORD,SEA,PNR_296108,2025-06-30,4,N,0,N,1
4,UA,1620,2025-08-01,ORD,SEA,PNR_296108,2025-06-30,4,Y,0,Y,0


In [189]:
score_cols = [col for col in df_master.columns if 'score' in col.lower()]
print(df_master[score_cols].sample(10))

      ground_time_pressure_score  load_factor_score  baggage_volume_score  \
1060                           1                  3                     0   
989                            1                  3                     0   
7145                           1                  3                     2   
4196                           1                  3                     0   
6293                           1                  3                     2   
916                            1                  0                     0   
5414                           1                  3                     0   
6912                           1                  3                     0   
5897                           4                  0                     0   
6828                           1                  3                     0   

      transfer_complexity_score  family_complexity_score  ssr_score  \
1060                          5                        0          7   
989       